In [1]:
#several extra dependencies are included because I have things I want to add
import numpy as np
import pandas as pd
import matplotlib as plt
import scipy as sp
import ipywidgets as widgets
import numexpr
import bottleneck
import time
import random

In [2]:
#take as input the minimum change in each value and the maximum
minimum_noise = (int(input("What minimum percentage noise would you like? ")))
maximum_noise = (int(input("What maximum percentage of noise would you like? ")))

What minimum percentage noise would you like? 0
What maximum percentage of noise would you like? 10


In [3]:
#key is the original data, template has the same structure (meaning same rows, but more columns) but has no data yet
key = pd.read_csv("FPMK_key.csv")
template = pd.read_csv("fpmk_template.csv")

In [4]:
def simulate_transcriptome_1(df1, df2):
    """Takes as input the template (first), and the key (second) dataframes"""
    results=[]
    #sets the max index we will access, need to subtract 1 because we index from zero
    l = df2.shape[1] -1
    #this is the function that is used to assign new values to the template
    def assign_value(item):
        #chooses random index from useful range of key
        random_num = random.randint(2, l)
        #selects value from key
        random_val = key_tup[random_num]
        #chooses value in range defined by noise
        random_noise = random.randint(minimum_noise, maximum_noise)
        #generates a random value so we can randomly either increase or decrease the value
        random_branch = random.random()
        if random_branch>0.5:
            positive_noise= 1 + (random_noise/100)
            item = random_val*positive_noise
        else:
            negative_noise = 1 - (random_noise/100)
            item = random_val * negative_noise
        return item
    #adds headers to results list
    results.append(tuple(list(df1)))
    #iterates over every row in template dataframe
    for row in df1.itertuples():
        row_list=[]
        row_list = list(row)
        row_num = int(row[0])
        #selects values from corresponding row in key
        key_tup=df2.iloc[row_num]
        #list comprehension for each row in template
        row_list = row_list[1:3] + [assign_value(i) for i in row_list[3:]]
        #changes each list to tuple to make dataframe creation easier
        row_tuple=tuple(row_list)
        results.append(row_tuple)
    results = pd.DataFrame(results)
    return results

In [5]:
start=time.time()
results = simulate_transcriptome_1(template, key)
print(results)
end=time.time()
print(end-start)

                   0            1          2          3           4    \
0               Symbol  Gene_Symbol      99115       9234          96   
1      ENSG00000000003       TSPAN6          0          0   0.0886988   
2      ENSG00000000005         TNMD          0          0           0   
3      ENSG00000000419         DPM1    3.37202    4.46824   0.0816563   
4      ENSG00000000457        SCYL3    1.45363   0.737039     1.29506   
5      ENSG00000000460     C1orf112   0.437689   0.424027    0.440494   
6      ENSG00000000938          FGR    29.2731    19.7099     34.5199   
7      ENSG00000000971          CFH          0          0    0.299035   
8      ENSG00000001036        FUCA2     10.359    9.06531     13.8697   
9      ENSG00000001084         GCLC    1.16001     2.0634     1.16001   
10     ENSG00000001167         NFYA    15.4591    15.2922     16.4336   
11     ENSG00000001460        STPG1   0.063327   0.483662    0.361244   
12     ENSG00000001461       NIPAL3   0.389307     

In [6]:
results.to_csv("results.csv", index=False)